In [6]:
import marvin

from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
from SpiffWorkflow.bpmn.parser import BpmnParser, BpmnValidator
import io

from core.bpmn_validator import validate_bpmn_string

import marvin
from pydantic import BaseModel
from typing import Optional
import xml
from result import Result, Ok, Err, is_ok, is_err

In [7]:

from pydantic import BaseModel


# Define your data structures
class XMLProcessObject(BaseModel):
    content: str
    type: str
    errors: list[str] = []
    warnings: list[str] = []
    
    
provider = OpenAIProvider(base_url='http://localhost:11434/v1', api_key='')
model = OpenAIModel(model_name='qwen3:4b',
                    provider=provider,
)


In [8]:
simple_process_description ="I take the customer order and pass it to the fulfillment team, then they package the order and ship it to the customer."

hi
hi


``` mermaid
graph LR;

    User --> |description|A[Bob 1]
    A --> |type|B[Bob 2]
    User --> |description or answers|B[Bob 2]
    PreviousRun --> |Current XML|B[Bob 2]
    B --> |XML Process Object|D{Validation Tool}
    D --> |Errors and Warnings|B
    D --> |Validated XML|C[Bob 3]
    C --> |Process| E[Process analysis tool]
    E --> |Analysis| C
    E --> |Refinement Questions| User

```

____

Chain of process analysis:

``` mermaid
graph LR;

A[XML Text]-->B{XML Parser}
B --> BErr[Parsing Errors]
B --> BOut[Parsed XML]
A1[Process Model Type]--> C{Process Model Parser}
BOut --> C
C --> CErr[Parsing Errors]
C --> COut[Parsed Process Model]
COut --> D{Process Model Validator}
D --> DErr[Validation Errors]
D --> DOut[Validated Process Model]
DOut --> E{Process Analysis Compiler}
E --> EErr[Analysis Compilation Errors]
E --> F[Process Analyzer]
F --> |Process Analysis|ProcessOpportunityDash[Process Opportunity Dashboard]
F --> |Process Analysis|Bob4[Process Consultant 4]
Bob4 -->|Biggest Remaining Questions| User

______

In [9]:
# show that bob 1 works
bob_1 = marvin.Agent(name="Bob_1", model=model, model_settings={'think':False},
                   instructions="""You are a expert business process modelling consultant. Your job is to look \
                       at at a business process and decide if it is best modelled as BPMN, DMN or CMMN. Please just\
                    respond with the appropriate model type, no explaination is needed.""")


answer = bob_1.say(simple_process_description)

print(answer)

assert answer == "BPMN"



Output()

[08/20/25 10:22:52] ERROR    marvin.engine.streaming: Error processing pydantic event              ]8;id=404547;file:///Users/marshallomoore/Desktop/ProjectsLocal/AgenticBobs/.venv/lib/python3.12/site-packages/marvin/engine/streaming.py\streaming.py]8;;\:]8;id=97145;file:///Users/marshallomoore/Desktop/ProjectsLocal/AgenticBobs/.venv/lib/python3.12/site-packages/marvin/engine/streaming.py#126\126]8;;\
                             PartStartEvent: list index out of range                                               

[08/20/25 10:22:53] ERROR    marvin.engine.streaming: Error processing pydantic event              ]8;id=992660;file:///Users/marshallomoore/Desktop/ProjectsLocal/AgenticBobs/.venv/lib/python3.12/site-packages/marvin/engine/streaming.py\streaming.py]8;;\:]8;id=457010;file:///Users/marshallomoore/Desktop/ProjectsLocal/AgenticBobs/.venv/lib/python3.12/site-packages/marvin/engine/streaming.py#126\126]8;;\
                             PartDeltaEvent: list index out of range                                               

                    ERROR    marvin.engine.streaming: Error processing pydantic event              ]8;id=903776;file:///Users/marshallomoore/Desktop/ProjectsLocal/AgenticBobs/.venv/lib/python3.12/site-packages/marvin/engine/streaming.py\streaming.py]8;;\:]8;id=315244;file:///Users/marshallomoore/Desktop/ProjectsLocal/AgenticBobs/.venv/lib/python3.12/site-packages/marvin/engine/streaming.py#126\126]8;;\
                             PartStartEvent: list index out of range                                               

BPMN


In [42]:


EMPTY_XML_TEMPLATES_MINIMAL = {
    "BPMN": """<?xml version="1.0" encoding="UTF-8"?>
<bpmn:definitions id="{definitions_id}" targetNamespace="{target_namespace}"
  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
  xmlns:bpmn="http://www.omg.org/spec/BPMN/20100524/MODEL"
  xmlns:bpmndi="http://www.omg.org/spec/BPMN/20100524/DI"
  xmlns:dc="http://www.omg.org/spec/DD/20100524/DC"
  xmlns:di="http://www.omg.org/spec/DD/20100524/DI">
  <!-- Add <bpmn:process>, <bpmn:collaboration>, etc. later -->
</bpmn:definitions>
""",

    "DMN": """<?xml version="1.0" encoding="UTF-8"?>
<definitions id="{definitions_id}" name="{definitions_name}" namespace="{target_namespace}"
  xmlns="https://www.omg.org/spec/DMN/20191111/MODEL/"
  xmlns:dmndi="https://www.omg.org/spec/DMN/20191111/DMNDI/"
  xmlns:di="http://www.omg.org/spec/DMN/20180521/DI/"
  xmlns:dc="http://www.omg.org/spec/DMN/20180521/DC/"
  xmlns:feel="https://www.omg.org/spec/DMN/20191111/FEEL/">
  <!-- Add <decision>, <inputData>, <businessKnowledgeModel>, etc. later -->
</definitions>
""",

    "CMMN": """<?xml version="1.0" encoding="UTF-8"?>
<cmmn:definitions id="{definitions_id}" targetNamespace="{target_namespace}"
  xmlns:cmmn="http://www.omg.org/spec/CMMN/20151109/MODEL"
  xmlns:cmmndi="http://www.omg.org/spec/CMMN/20151109/CMMNDI"
  xmlns:dc="http://www.omg.org/spec/DD/20100524/DC"
  xmlns:di="http://www.omg.org/spec/DD/20100524/DI">
  <!-- Add <cmmn:case> later -->
</cmmn:definitions>
""",

    "ArchiMate": """<?xml version="1.0" encoding="UTF-8"?>
<model xmlns="http://www.opengroup.org/xsd/archimate/3.0/"
  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
  identifier="{model_id}" name="{model_name}">
  <metadata>
    <schema>{schema_version}</schema>
  </metadata>
  <!-- Add <elements>, <relationships>, <organizations>, <properties> later -->
</model>
""",
}

# (Optional) Example of just the global setup values you might pass:
EXAMPLE_GLOBAL_INPUTS = {
    "BPMN": {
        "definitions_id": "def_bpmn_empty",
        "target_namespace": "urn:example:bpmn",
    },
    "DMN": {
        "definitions_id": "def_dmn_empty",
        "definitions_name": "EmptyDMN",
        "target_namespace": "urn:example:dmn",
    },
    "CMMN": {
        "definitions_id": "def_cmmn_empty",
        "target_namespace": "urn:example:cmmn",
    },
    "ArchiMate": {
        "model_id": "archi_model_empty",
        "model_name": "EmptyModel",
        "schema_version": "3.0",
    },
}

# Render one (example):
# EMPTY_XML_TEMPLATES_MINIMAL["BPMN"].format(**EXAMPLE_GLOBAL_INPUTS["BPMN"])


def get_empty_process_model(model_type):
    if model_type not in EMPTY_XML_TEMPLATES_MINIMAL:
        raise ValueError(f"Unknown model type: {model_type}")
    return EMPTY_XML_TEMPLATES_MINIMAL[model_type], EXAMPLE_GLOBAL_INPUTS[model_type]

In [14]:
# show getting the empty process model
empty_proc, examples = get_empty_process_model(answer)

In [ ]:
#show bob2 at work 

bob_2 = marvin.Agent(name="Bob_2", model=model, model_settings={'think':False},
                   instructions="""You are an expert business process modelling consultant. Your job is to look at \
                       a current XML diagram of a business process, along with a english language description of a business process\
                       and to output a new valid diagram that reflects the business process described.""",
                       tools=[validate_bpmn_string]
)




t1 = marvin.Task('Make the necessary changes to the XML diagram to align it with the described business process', agents=[bob_2],
            context={"description": simple_process_description, "xml_process": empty_proc},
)


NameError: name 'marvin' is not defined

In [19]:
output = t1.run()

In [21]:
print(output)

<?xml version="1.0" encoding="UTF-8"?>
<bpmn:definitions id="process_definitions" targetNamespace="http://example.com/processes" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:bpmn="http://www.omg.org/spec/BPMN/20100524/MODEL">
  <bpmn:process id="process_1" name="Order Fulfillment Process">
    <bpmn:startEvent id="start_event_1" name="Customer Order Received"/>
    <bpmn:task id="task_1" name="Pass order to fulfillment team"/>
    <bpmn:task id="task_2" name="Package order"/>
    <bpmn:task id="task_3" name="Ship order to customer"/>
    <bpmn:endEvent id="end_event_1" name="Order Shipped"/>
    <bpmn:sequenceFlow id="flow_1" sourceRef="start_event_1" targetRef="task_1"/>
    <bpmn:sequenceFlow id="flow_2" sourceRef="task_1" targetRef="task_2"/>
    <bpmn:sequenceFlow id="flow_3" sourceRef="task_2" targetRef="task_3"/>
    <bpmn:sequenceFlow id="flow_4" sourceRef="task_3" targetRef="end_event_1"/>
  </bpmn:process>
</bpmn:definitions>


In [ ]:
valid = BpmnValidator()

parser = BpmnParser(validator=valid)

bytes_xml = output.encode("UTF-8")
bytes_io = io.BytesIO(bytes_xml)
parser.add_bpmn_io(bytes_io)

In [ ]:
par

In [ ]:

# Define your data structures
class XMLProcessObject(BaseModel):
    content: str
    type: str
    errors: list[str] = []
    warnings: list[str] = []
    
    
provider = OpenAIProvider(base_url='http://localhost:11434/v1', api_key='')
model = OpenAIModel(model_name='qwen3:4b',
                    provider=provider,
)


    
bob_1 = marvin.Agent(name="Bob_1", model=model, model_settings={'think':False},
                   instructions="""You are a expert business process modelling consultant. Your job is to look \
                       at at a business process and decide if it is best modelled as BPMN, DMN or CMMN. Please just\
                    respond with the appropriate model type, no explaination is needed.""")


bob_2 = marvin.Agent(name="Bob_2", model=model, model_settings={'think':False},
                   instructions="""You are an expert business process modelling consultant. Your job is to look at \
                       a current XML diagram of a business process, along with a english language description of a business process\
                       and to output required changes to the XML diagram to better align it with the described process.""")


# Create specialized agents
bob1 = marvin.Agent(
    name="Bob 1",
    instructions="Convert user descriptions into process types"
)

bob2 = marvin.Agent(
    name="Bob 2", 
    instructions="Generate XML Process Objects from descriptions and types"
)

bob3 = marvin.Agent(
    name="Bob 3",
    instructions="Perform process analysis and generate refinement questions"
)

# Create validation tool
def validation_tool(xml_obj: XMLProcessObject) -> XMLProcessObject:
    """Validate XML Process Object and return errors/warnings"""
    # Your validation logic here
    validated_obj = xml_obj.model_copy()
    # Add validation results
    return validated_obj

def process_analysis_tool(xml_obj: XMLProcessObject) -> dict:
    """Analyze the process and return analysis results"""
    # Your analysis logic here
    return {"analysis": "Process analysis results"}





with marvin.Thread() as thread:
    # Check if process type exists in thread context
    if not hasattr(thread, '_process_type'):
        thread._process_type = marvin.run(
            "Convert description to process type",
            agents=[bob1],
            context = {"description": simple_process_description},
            thread=thread
        )
        
        
    if not hasattr(thread, '_xml_process'):
        thread._xml_process, example_inputs = get_empty_process_model(thread._process_type, EMPTY_XML_TEMPLATES_MINIMAL)

    # Use the cached process type
    xml_process = marvin.run(
        "Generate XML Process Object", 
        agents=[bob2],
        context={"type": thread._process_type},
        thread=thread
    )
    
    # Step 3: Validation loop (Bob 2 can retry based on validation)
    while xml_process.errors:
        xml_process = marvin.run(
            "Fix XML Process Object based on validation errors",
            agents=[bob2],
            result_type=XMLProcessObject,
            tools=[validation_tool],
            context={"errors": xml_process.errors},
            thread=thread
        )
    
    # Step 4: Bob 3 performs analysis
    analysis = marvin.run(
        "Analyze the validated XML process",
        agents=[bob3],
        tools=[process_analysis_tool],
        context={"xml_process": xml_process},
        thread=thread
    )
    
    # Step 5: Generate refinement questions for user
    questions = marvin.run(
        "Generate refinement questions based on analysis",
        agents=[bob3],
        result_type=list[str],
        context={"analysis": analysis},
        cli=True,  # Enable CLI interaction with user
        thread=thread
    )